In [1]:
!pip install aiohttp
!pip install datasets
!pip install jiwer
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
from pydub import AudioSegment
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras import layers
import keras
from tqdm import tqdm
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Reshape, Dense, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from jiwer import wer
from contractions import fix

import warnings
warnings.filterwarnings("ignore")


In [3]:
# The dataset used is the common voice dataset from Mozilla. It is readily available on Kaggle

train_X_path = "/kaggle/input/common-voice/cv-valid-train"
train_y_path = "/kaggle/input/common-voice/cv-valid-train.csv"

test_X_path = "/kaggle/input/common-voice/cv-valid-test"
test_y_path = "/kaggle/input/common-voice/cv-valid-test.csv"

In [4]:
X = []
y = []

df = pd.read_csv(train_y_path)
x=0
for index, row in df.iterrows():
    X.append(os.path.join(train_X_path, row['filename']))
    y.append(row['text'])


df = pd.read_csv(test_y_path)
x=0
for index, row in df.iterrows():
    X.append(os.path.join(test_X_path, row['filename']))
    y.append(row['text'])

In [5]:
print(len(X))
print(len(y))

199771
199771


In [6]:
# Checking if any text has a digit

count = 0

for i in y:
    if(any(char.isdigit() for char in i) ):
        count = count + 1

print(f"Number of samples with digits=",count)

Number of samples with digits= 0


In [85]:
def count_samples_max_len(length):

    count = 0
    
    for i in y:
        if(len(i)<length):
            count = count + 1

    return count


print(count_samples_max_len(30)) # Number of samples with length less than 80
print(count_samples_max_len(50)) # Number of samples with length less than 80
print(count_samples_max_len(80)) # Number of samples with length less than 80
print(count_samples_max_len(100))  # Number of samples with length less than 100
print(count_samples_max_len(120))  # Number of samples with length less than 120
print(count_samples_max_len(200))  # Number of samples with length less than 200

43336
113416
182328
182328
182328
182328


In [8]:
def remove_samples_greater_than_max_length(maxlen):
    X_new = []
    y_new = []

    for i in range(0, len(X)):
        if(len(y[i])<maxlen):
            X_new.append(X[i])
            y_new.append(y[i])

    return X_new, y_new

maxlen = 80
X, y  = remove_samples_greater_than_max_length(maxlen)

In [9]:
print(len(X))
print(len(y))

182328
182328


In [10]:
print(X[0])
print(y[0])

/kaggle/input/common-voice/cv-valid-train/cv-valid-train/sample-000000.mp3
learn to recognize omens and follow them the old king had said


In [11]:
# Mappings for text
characters = [x for x in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']]

# Mapping characters to integers
char_to_int = keras.layers.StringLookup(vocabulary = characters, oov_token = "")
# Reverse mapping fr integers to characters
int_to_char = keras.layers.StringLookup(vocabulary= char_to_int.get_vocabulary(), oov_token = "", invert = True)

print(
    f"The vocabulary is: {char_to_int.get_vocabulary()} "
    f"(size ={char_to_int.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' '] (size =28)


In [12]:
max_length = 80

def process_text(sample):
    sample = sample.lower()
    sample = tf.strings.unicode_split(sample, input_encoding="UTF-8")
    sample = char_to_int(sample)
    padded_sample = tf.pad(
        sample,
        paddings=[[0, max_length - tf.shape(sample)[0]]],  # Pad only the sequence axis
        mode="CONSTANT",
        constant_values=char_to_int("")  # Pad with the index of "" (usually 0)
    )
    return padded_sample

print(process_text(y[0]))

tf.Tensor(
[12  5  1 18 14 27 20 15 27 18  5  3 15  7 14  9 26  5 27 15 13  5 14 19
 27  1 14  4 27  6 15 12 12 15 23 27 20  8  5 13 27 20  8  5 27 15 12  4
 27 11  9 14  7 27  8  1  4 27 19  1  9  4  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0], shape=(80,), dtype=int64)


In [13]:
def process_audio(audio, sr, target_length_ms=1000, n_mfcc=40,
                  window_ms=25, hop_ms=10, fix_length=True):
    """
    Process audio to fixed-length normalized MFCCs.

    Args:
        audio: 1D numpy array (time series)
        sr: Sample rate (Hz)
        target_length_ms: Target duration in milliseconds (default: 1000ms)
        n_mfcc: Number of MFCC coefficients (default: 40)
        window_ms: STFT window length (ms) (default: 25ms)
        hop_ms: STFT hop length (ms) (default: 10ms)
        fix_length: Pad/trim audio to fixed length (default: True)

    Returns:
        Normalized MFCCs with shape (n_mfcc, fixed_time_steps)
    """
    # Convert ms to samples
    target_samples = int(target_length_ms * sr / 1000)
    window_samples = int(window_ms * sr / 1000)
    hop_samples = int(hop_ms * sr / 1000)

    # 1. Force fixed audio length
    if fix_length:
        if len(audio) > target_samples:
            audio = audio[:target_samples]  # Trim
        else:
            padding = target_samples - len(audio)
            audio = np.pad(audio, (0, padding), mode='constant')  # Pad with zeros

    # 2. Extract MFCCs
    mfccs = librosa.feature.mfcc(
        y=audio,
        sr=sr,
        n_mfcc=n_mfcc,
        n_fft=window_samples,
        hop_length=hop_samples,
        center=True  # Important for short files
    )

    # 3. Calculate fixed expected time steps
    if fix_length:
        expected_steps = int(target_length_ms / hop_ms) + 1
        if mfccs.shape[1] < expected_steps:
            # Pad MFCCs if shorter than expected
            pad_width = ((0, 0), (0, expected_steps - mfccs.shape[1]))
            mfccs = np.pad(mfccs, pad_width, mode='constant')
        elif mfccs.shape[1] > expected_steps:
            # Trim if longer (shouldn't happen with fixed audio)
            mfccs = mfccs[:, :expected_steps]

    # 4. Normalize
    mfcc_mean = np.mean(mfccs, axis=1, keepdims=True)
    mfcc_std = np.std(mfccs, axis=1, keepdims=True)
    mfccs_normalized = (mfccs - mfcc_mean) / (mfcc_std + 1e-6)

    return mfccs_normalized

In [14]:
def process_sample(audio, sr, text):
  audio = process_audio(audio, sr)
  text = process_text(text)
  return [audio, text]

In [15]:
def process_dataset():
    df = []
    for i in range(len(X)):
        path = X[i]
        text = y[i]
        audio, sr = audio, sr = librosa.load(path, sr=16000, mono=True, duration=5.0)
        df.append(process_sample(audio, sr, text))
        if(i>10000):
            break

    return df

#dataset = process_dataset()

In [16]:
%%time

dataset = process_dataset()

CPU times: user 5min 29s, sys: 9.66 s, total: 5min 39s
Wall time: 4min 44s


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_model(input_dim, output_dim, rnn_layers=4, rnn_units=128):
    """Build a CTC-based speech recognition model."""
    # Input: Mel spectrograms (shape: [batch, time, n_mels])
    input_spectrogram = layers.Input(shape=input_dim, name="input_spectrogram")
    
    # CNN for feature extraction
    x = layers.Conv1D(64, 3, activation="relu", padding="same")(input_spectrogram)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)

    
    # RNN (Bi-LSTM for sequence modeling)
    for _ in range(rnn_layers):
        x = layers.Bidirectional(
            layers.LSTM(rnn_units, return_sequences=True, dropout=0.2)
        )(x)
    
    # Output layer
    x = layers.Dense(rnn_units, activation="relu")(x)
    output = layers.Dense(output_dim + 1, activation="softmax")(x)  # +1 for CTC blank token

    model = Model(input_spectrogram, output)
    return model

In [18]:
X_train_mel = []
y_train_padded = []

for i in dataset:
    X_train_mel.append(i[0])
    y_train_padded.append(i[1])

In [19]:
X_train_mel, X_test_mel, y_train_padded, y_test_padded = train_test_split(
    X_train_mel, y_train_padded,
    train_size=0.8,  
    shuffle=True,
    random_state=42, 
)

X_test_mel, X_val_mel, y_test_padded, y_val_padded = train_test_split(
    X_test_mel, y_test_padded,
    train_size=0.5,  
    shuffle=True,
    random_state=42, 
)

In [20]:
print(len(X_train_mel))
print(len(y_train_padded))
print(len(X_val_mel))
print(len(y_val_padded))
print(len(X_test_mel))
print(len(y_test_padded))

8001
8001
1001
1001
1000
1000


In [22]:
X_train_mel = np.array(X_train_mel)
y_train_padded = np.array(y_train_padded)
X_val_mel = np.array(X_val_mel)
y_val_padded = np.array(y_val_padded)
X_test_mel = np.array(X_test_mel)
y_test_padded = np.array(y_test_padded)

In [23]:
def ctc_loss(y_true, y_pred):
    # Get batch size from predictions (shape should be [batch, time, num_classes])
    batch_size = tf.shape(y_pred)[0]
    time_steps = tf.shape(y_pred)[1]
    
    # Input length (same for all samples in batch)
    input_length = tf.fill([batch_size], time_steps)
    
    # Label length (from y_true shape, assuming shape [batch, max_label_len])
    label_length = tf.fill([batch_size], tf.shape(y_true)[1])
    
    # Ensure y_true is int32 (required by CTC)
    y_true = tf.cast(y_true, tf.int32)
    
    # Compute CTC loss
    ctc_loss_value = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=tf.fill([tf.shape(y_pred)[0]], tf.shape(y_true)[1]),
        logit_length=tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1]),
        logits_time_major=False
    )
    # Average loss per time step
    return ctc_loss_value / tf.cast(tf.shape(y_pred)[1], tf.float32)

In [24]:
# Parameters
input_dim = (40, 101)  # Number of Mel bands (n_mels)
output_dim = len(char_to_int.get_vocabulary())  # Vocabulary size

# Build model
model = build_model(input_dim, output_dim)
model.compile(optimizer="adam", loss=ctc_loss)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_spectrogram (InputLayer)       │ (None, 40, 101)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 40, 64)              │          19,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 40, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 20, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 20, 256)             │         197,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 20, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 20, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 20, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20, 128)             │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20, 29)              │           3,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,436,701 (5.48 MB)

 Trainable params: 1,436,573 (5.48 MB)

 Non-trainable params: 128 (512.00 B)

In [25]:
# Train
history = model.fit(
    x=X_train_mel,  # Preprocessed Mel spectrograms (shape: [batch, time, n_mels])
    y=y_train_padded,  # Padded integer-encoded text labels
    validation_data=(X_val_mel, y_val_padded),
    batch_size=32,
    epochs=10,
)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 28s 63ms/step - loss: 37.4054 - val_loss: 37.3665
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - loss: 37.3679 - val_loss: 37.3665
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3678 - val_loss: 37.3665
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 15s 58ms/step - loss: 37.3680 - val_loss: 37.3665
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3677 - val_loss: 37.3665
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3676 - val_loss: 37.3665
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3677 - val_loss: 37.3665
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3678 - val_loss: 37.3665
Epoch 9/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - loss: 37.3676 - val_loss: 37.3665
Epoch 10/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - loss: 37.3679 - val_loss: 37.3665


In [47]:
def build_enhanced_model(input_dim, output_dim, rnn_layers=8, rnn_units=256):
    input_spectrogram = layers.Input(shape=input_dim, name="input_spectrogram")
    
    # Enhanced CNN block
    x = layers.Conv1D(64, 3, activation="relu", padding="same")(input_spectrogram)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    
    #residual = layers.Conv1D(128, 1)(x)  
    x = layers.Conv1D(128, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Add()([x, residual])

    #residual = layers.Conv1D(256, 1)(x)  
    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Add()([x, residual])
    
    # Hierarchical RNN
    for _ in range(rnn_layers//2):
        x = layers.Bidirectional(layers.LSTM(rnn_units, return_sequences=True, dropout=0.2))(x)
    
    for _ in range(rnn_layers//2):
        x = layers.Bidirectional(layers.LSTM(rnn_units*2, return_sequences=True, dropout=0.3))(x)

    # Multi-head self-attention
    x = layers.MultiHeadAttention(num_heads=8, key_dim=rnn_units//4)(x, x)
    x = layers.LayerNormalization()(x)  # Helps with training stability
    
    # Attention mechanism
    # attention = layers.Attention()([x, x])
    # x = layers.Concatenate()([x, attention])
    
    # Output layers
    x = layers.Dense(rnn_units*2, activation="relu")(x)
    x = layers.Dense(rnn_units*2, activation="relu")(x)
    output = layers.Dense(output_dim + 1, activation="linear")(x)

    return Model(input_spectrogram, output)

In [51]:
print(output_dim)

28


In [56]:
def ctc_loss(y_true, y_pred):
    # Get batch dimensions
    batch_size = tf.shape(y_pred)[0]
    time_steps = tf.shape(y_pred)[1]
    
    # Input length (all sequences use full timesteps)
    input_length = tf.fill([batch_size], time_steps)
    
    # Label length (count non-padding tokens - assumes padding is 0)
    label_length = tf.reduce_sum(
        tf.cast(tf.not_equal(y_true, 0)), axis=1)
    
    # Convert y_true to int32 (required by CTC)
    y_true = tf.cast(y_true, tf.int32)
    
    # Compute CTC loss
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=input_length,
        logits_time_major=False
    )
    
    # Return mean loss across batch
    return tf.reduce_mean(loss)

In [62]:
def ctc_loss(y_true, y_pred):
    # Get batch dimensions
    batch_size = tf.shape(y_pred)[0]
    time_steps = tf.shape(y_pred)[1]
    
    # Input length (all sequences use full timesteps)
    input_length = tf.fill([batch_size], time_steps)
    
    # Label length - count non-zero elements (assuming 0 is padding)
    # First convert y_true to float32 for comparison
    y_true_float = tf.cast(y_true, tf.float32)
    # Create mask of non-padding elements
    mask = tf.not_equal(y_true_float, 0.0)
    # Count non-padding elements for each sequence
    label_length = tf.reduce_sum(
        tf.cast(mask, tf.int32),
        axis=1
    )
    
    # Convert y_true to int32 (required by CTC)
    y_true = tf.cast(y_true, tf.int32)
    
    # Compute CTC loss
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=input_length,
        logits_time_major=False,
        blank_index=-1  # Important if you're using 0 as padding
    )
    
    # Return mean loss across batch
    return tf.reduce_mean(loss)

In [63]:
# Define parameters
num_classes = 29  # Adjust to your actual number of classes
max_label_length = 5
time_steps = 50

# Create test data
y_true = tf.constant([
    [1, 2, 3, 0, 0],  # First sample: 3 real tokens, 2 padding
    [4, 5, 0, 0, 0]   # Second sample: 2 real tokens, 3 padding
], dtype=tf.int32)

y_pred = tf.random.normal(
    shape=[2, time_steps, num_classes],  # [batch, time, classes]
    dtype=tf.float32
)

# Test the function
test_loss = ctc_loss(y_true, y_pred)
print("Test CTC loss:", test_loss.numpy())

Test CTC loss: 163.37225


In [66]:
# Parameters
input_dim = (40, 101)  # Number of Mel bands (n_mels)
output_dim = len(char_to_int.get_vocabulary())  # Vocabulary size

# Build model
model = build_enhanced_model(input_dim, output_dim)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),  # Start with 1e-4
    loss=ctc_loss
)

model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_spectrogram         │ (None, 40, 101)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_26 (Conv1D)        │ (None, 40, 64)         │         19,456 │ input_spectrogram[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_16    │ (None, 40, 64)         │            256 │ conv1d_26[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_6           │ (None, 20, 64)         │              0 │ batch_normalization_1… │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_28 (Conv1D)        │ (None, 20, 128)        │          8,320 │ max_pooling1d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_17    │ (None, 20, 128)        │            512 │ conv1d_28[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_27 (Conv1D)        │ (None, 20, 128)        │          8,320 │ max_pooling1d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_10 (Add)              │ (None, 20, 128)        │              0 │ batch_normalization_1… │
│                           │                        │                │ conv1d_27[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_30 (Conv1D)        │ (None, 20, 256)        │         33,024 │ add_10[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 20, 256)        │          1,024 │ conv1d_30[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_29 (Conv1D)        │ (None, 20, 256)        │         33,024 │ add_10[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_11 (Add)              │ (None, 20, 256)        │              0 │ batch_normalization_1… │
│                           │                        │                │ conv1d_29[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_44          │ (None, 20, 512)        │      1,050,624 │ add_11[0][0]           │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_45          │ (None, 20, 512)        │      1,574,912 │ bidirectional_44[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_46          │ (None, 20, 512)        │      1,574,912 │ bidirectional_45[0][0] │
│ (Bidirectional)      

 Total params: 31,868,445 (121.57 MB)

 Trainable params: 31,867,549 (121.57 MB)

 Non-trainable params: 896 (3.50 KB)

In [67]:
history = model.fit(
    x=X_train_mel,
    y=y_train_padded,
    batch_size=32,  # Start small
    epochs=50,
    validation_data=(X_val_mel, y_val_padded),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
    ]
)

Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 50s 131ms/step - loss: 699.2029 - val_loss: 684.2969 - learning_rate: 0.0100
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 122ms/step - loss: 697.4433 - val_loss: 684.2969 - learning_rate: 0.0100
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - loss: 695.3046 - val_loss: 684.2969 - learning_rate: 0.0100
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 122ms/step - loss: 698.4875 - val_loss: 684.2969 - learning_rate: 0.0100
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - loss: 697.1153 - val_loss: 684.2969 - learning_rate: 0.0050
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 122ms/step - loss: 699.6515 - val_loss: 684.2969 - learning_rate: 0.0050


In [81]:
import tensorflow as tf

def debug_ctc_loss(y_true, y_pred, blank_index=0):
    label_length = tf.reduce_sum(tf.cast(y_true != blank_index, tf.int32), axis=1)
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1]),
        blank_index=blank_index,  # Critical!
        logits_time_major=False
    )
    return tf.reduce_mean(loss)

In [82]:
y_true = tf.constant([[1, 2, 0]], dtype=tf.int32)  # Padding=0
y_pred = tf.constant(
    [[[ -5, 10, -5, -5],   # Peak at class 1
      [ -5, -5, 10, -5],   # Peak at class 2
      [ 10, -5, -5, -5]]], # Peak at blank (0)
    dtype=tf.float32
)
loss = debug_ctc_loss(y_true, y_pred, blank_index=0)
print("Test 1 Loss (should be ~0.001):", loss.numpy())

Test 1 Loss (should be ~0.001): 2.503395e-06


In [83]:
y_true = tf.constant([[1, 3, 0]], dtype=tf.int32)  # Truth: [1, 3]
y_pred = tf.constant(
    [[[ -5, -5, 10, -5],   # Peak at class 2 (wrong)
      [ -5, 10, -5, -5],   # Peak at class 1 (wrong)
      [ 10, -5, -5, -5]]], # Peak at blank
    dtype=tf.float32
)
loss = debug_ctc_loss(y_true, y_pred, blank_index=0)
print("Test 2 Loss (should be ~15-25):", loss.numpy())

Test 2 Loss (should be ~15-25): 28.90139


In [44]:
output = model.predict(X_test_mel)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
